# 프로젝트1 : 주문서 정리 

월별로 시트가 구분되어 있는 매출 데이터를 요약하여 정리한다.  

데이터 병합 > 포맷 통일 및 변수 추가 > 월별 매출 추이 파악 > 조건에 따른 판매 통계 분석 > 충성 고객 파악  


In [1]:
import pandas as pd
import os
os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/1. 데이터 핸들링/데이터")

## Step 1. 데이터 병합하기

하나의 데이터가 12개의 시트로 구분되어 있어, 효과적인 분석을 위해 통합해야 한다. 또한, 분석에 불필요한 부분(셀)이 있어 이를 제거해야 한다.  

### 1. 시트명을 정의하여 각 시트명을 바탕으로 데이터를 불러오도록 한다.  


In [2]:
# 시트명

sheet_names = [str(i+1) + '월' for i in range(12)]
sheet_names

['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

### 2. 데이터를 시트별로 불러와서 한 데이터 프레임에 모은다

In [3]:
# concat 이용
merged_df = pd.DataFrame()
for sn in sheet_names:
    df = pd.read_excel('월별매출데이터.xlsx', sheet_name = sn, 
                       skiprows = range(6)).iloc[:, 1:]
    merged_df = pd.concat([merged_df, df], axis = 0, ignore_index = True)

merged_df['일자']

0          2018.1.1
1          2018.1.1
2          2018.1.1
3          2018.1.1
4          2018.1.1
            ...    
18224    2018-12-31
18225    2018-12-31
18226    2018-12-31
18227    2018-12-31
18228    2018-12-31
Name: 일자, Length: 18229, dtype: object

In [4]:
merged_df.head()

,일자,지점,품명,수량,주문인 ID,수령 주소,주문 상태,결제 수단
0,2018.1.1,지점1,제품B,3,C-168,서울특별시 동작구 흑석동,주문완료,인터넷뱅킹
1,2018.1.1,지점2,제품F,8,C-87,서울특별시 서대문구 신촌동,배송완료,신용카드
2,2018.1.1,지점4,제품B,2,C-158,서울특별시 종로구 종로5가,배송완료,휴대폰결제
3,2018.1.1,지점3,제품D,7,C-307,서울특별시 서대문구 냉천동,주문완료,휴대폰결제
4,2018.1.1,지점2,제품E,9,C-342,서울특별시 종로구 묘동,배송완료,인터넷뱅킹


## Step 2. 포맷 통일 및 변수 추가



### 문자열 검색을 통해 날짜변수 포맷 통일
일자 컬럼이 .과 - 가 섞여있는 상태로 통일해야 한다.  
연도가 같으니 월 일만 추출할 수도 있다.  

- str.replace를 사용하여 '.'을 '-'로 변경 : 문자열 검색  

 
```FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.```

In [6]:
merged_df['일자'] = merged_df['일자'].str.replace('.', '-')
merged_df['일자']

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_20351/1296823592.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df['일자'] = merged_df['일자'].str.replace('.', '-')


0          2018-1-1
1          2018-1-1
2          2018-1-1
3          2018-1-1
4          2018-1-1
            ...    
18224    2018-12-31
18225    2018-12-31
18226    2018-12-31
18227    2018-12-31
18228    2018-12-31
Name: 일자, Length: 18229, dtype: object

### 제품별 가격정보 불러오기

원래 데이터프레임에는 제품 이름만 있고 가격이 없다.  
$주문금액 = 수량 \times 가격 \times 1.1 (부가세)$

매출을 살피기위해 제품별 가격정보를 불러와서, 원래 데이터프레임이 이를 참조하도록 한다.  

제품에 따른 가격 정보는 제품별_가격정보.xlsx에 정의되어있다.

---

#### 새로운 문법(중요)

- `Series.to_dict()` : Series의 index를 key로, data를 value로 갖는 사전으로 변환. `_list`, `_values` 도 있다.  
- `Series.replace(dict)` : Series에 있는 값 가운데 dict의 key의 값이 있으면 대응되는 value로 변환.  
replace는 딕셔너리를 입력받기 때문에 to_dict를 사용하며, 있는 시리즈를 기반으로 무언가 값을 바꾸거나 입력하기 위해서 사용한다.  
- `DataFrame.T` : 전치행렬 반환  
- `DataFrame['new_col'] = values`(연산결과): 새로운 컬럼 추가  


In [14]:
price_df = pd.read_excel('제품별_가격정보.xlsx')
price_df

,제품,제품A,제품B,제품C,제품D,제품E,제품F
0,가격,20000,5000,10000,8000,30000,15000


In [17]:
# 제품의 이름이 인덱스로 오도록 행렬 전치
# index: 제품, 0: 가격
price_df = price_df.T
price_df

,0
제품,가격
제품A,20000
제품B,5000
제품C,10000
제품D,8000
제품E,30000
제품F,15000


인덱스로 되어있는 제품 이름을 통해 필요한 부분만 잘라내고 그를 딕셔너리로 만들어준다.  

새로운 데이터프레임의 컬럼명이 0임

In [18]:
price_df.loc['제품A':'제품F', 0]

제품A    20000
제품B     5000
제품C    10000
제품D     8000
제품E    30000
제품F    15000
Name: 0, dtype: object

In [19]:
price_dict = price_df.loc['제품A':'제품F', 0].to_dict()
price_dict

{'제품A': 20000,
 '제품B': 5000,
 '제품C': 10000,
 '제품D': 8000,
 '제품E': 30000,
 '제품F': 15000}

### 주문금액 컬럼 만들기

위에서 만든 딕셔너리로 제품 자체의 가격 컬럼과 주문금액 컬럼을 만든다.  
가격변수가 필요한 상황이 아니라면 가격변수 정의할 계산 자체를 주문금액 컬럼 만드는데 써도 좋다.  

In [20]:
merged_df['품명'].unique()

array(['제품B', '제품F', '제품D', '제품E', '제품C', '제품A'], dtype=object)

In [21]:
merged_df['가격'] = merged_df['품명'].replace(price_dict)
merged_df[['품명', '가격']]

,품명,가격
0,제품B,5000
1,제품F,15000
2,제품B,5000
3,제품D,8000
4,제품E,30000
...,...,...
18224,제품F,15000
18225,제품D,8000
18226,제품D,8000
18227,제품B,5000


In [22]:
# ndarray 브로드캐스팅 + 유니버설 함수
merged_df['주문 금액'] = merged_df['수량'] * merged_df['가격'] * 1.1 

In [23]:
merged_df.head()

,일자,지점,품명,수량,주문인 ID,수령 주소,주문 상태,결제 수단,가격,주문 금액
0,2018-1-1,지점1,제품B,3,C-168,서울특별시 동작구 흑석동,주문완료,인터넷뱅킹,5000,16500.0
1,2018-1-1,지점2,제품F,8,C-87,서울특별시 서대문구 신촌동,배송완료,신용카드,15000,132000.0
2,2018-1-1,지점4,제품B,2,C-158,서울특별시 종로구 종로5가,배송완료,휴대폰결제,5000,11000.0
3,2018-1-1,지점3,제품D,7,C-307,서울특별시 서대문구 냉천동,주문완료,휴대폰결제,8000,61600.0
4,2018-1-1,지점2,제품E,9,C-342,서울특별시 종로구 묘동,배송완료,인터넷뱅킹,30000,297000.0


## Step 3. 월별 매출 추이 파악
